# Get Councillors

Récupérer la liste actuelle des parlementaires en JSON grâce à l’API du parlement

http://ws-old.parlament.ch/

«Pour le moment, les nouveaux services web ne sont pas encore disponibles. Nous sommes cependant certains que nous pourrons remédier à cette situation d’ici au troisième trimestre 2019.»

Votes:
http://ws-old.parlament.ch/votes/councillors

http://ws-old.parlament.ch/votes/councillors/3055?legislativePeriodFiter=662

In [1]:
import pandas as pd
import requests
from time import sleep
pd.set_option('display.max_colwidth', 999)

In [38]:
# Choix de la législature
legislative_period = 45

In [39]:
rows = []
page_number = 1
delay = 0.1
while True:
    print('', end='=')
    try:
        response = requests.get(f'http://ws-old.parlament.ch/councillors/historic?legislativePeriodFromFilter={legislative_period}&format=json&lang=fr&pageNumber={page_number}', headers={'Accept': 'application/json', 'Accept-Language': 'fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5'})
        data = response.json()
        rows.extend(data)
        if data[-1].get('hasMorePages') == False:
            print('No more pages.')
            break
    except Exception as e:
        print('Error', response.status_code, '/', e, 'probably end of pagination')
        break
    page_number += 1
    sleep(delay)

======No more pages.


In [40]:
df_councillors = pd.DataFrame(rows)

In [41]:
def get_grade(councillor_id):
    grade = 'not found'
    try:
        response = requests.get(f'http://ws-old.parlament.ch/councillors/{councillor_id}?format=json&lang=fr', headers={'Accept': 'application/json', 'Accept-Language': 'fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5'})
        grade = response.json().get('militaryGrade')
    except:
        print('error')
        grade = 'http error' + str(response.status_code)
    return grade

df_councillors['grade'] = df_councillors['id'].apply(get_grade)

In [42]:
# Législature 45
df_councillors['grade'].value_counts()

Soldat                    18
Premier-lieutenant        18
Capitaine                 14
Major                     10
Appointé                   9
Colonel                    6
Sergent                    5
Caporal                    4
Fourrier                   3
Lieutenant-colonel         3
Adjudant sous-officier     1
Sous-officier              1
--                         1
Brigadier                  1
Name: grade, dtype: int64

**Législature 51**

```
Soldat                       28
Appointé                     11
Capitaine                     8
--                            5
Premier-lieutenant            4
Major                         4
Sergent                       3
Colonel                       2
Fourrier aD                   2
Caporal                       2
Fourrier                      2
Soldat aD                     2
Capitaine aD                  1
Officier spécialisé (maj)     1
Appointé-chef                 1
Lieutenant-colonel            1
-                             1
```

In [6]:
df_councillors['council'] = df_councillors['council'].apply(lambda x: x['abbreviation'])

In [7]:
df_councillors.to_csv('data/councillors/councillors_2019.csv', index=False)